In [16]:
!git clone https://github.com/jgbeni/BirdPics.git
!mv BirdPics/utils .
!rm -r BirdPics
!ls -l

Cloning into 'BirdPics'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 74 (delta 29), reused 37 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 6.80 MiB | 13.84 MiB/s, done.
Resolving deltas: 100% (29/29), done.
mv: cannot move 'BirdPics/utils' to './utils': Directory not empty
total 12
drwx------ 5 root root 4096 Nov  6 13:58 drive
drwxr-xr-x 1 root root 4096 Nov  4 14:36 sample_data
drwxr-xr-x 3 root root 4096 Nov  6 13:57 utils


In [17]:
import utils.data_preprocessing as dp
import numpy as np
import h5py
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import os
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive',force_remount=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

Mounted at /content/drive


In [18]:
dir = '/content/drive/MyDrive/BirdPics'
os.makedirs(dir+'/models',exist_ok=True)

In [19]:
f = h5py.File(dir+'/data/bird_data.hdf5', "r")

In [20]:
X_train,Y_train = f['train']['X'],np.copy(f['train']['Y'])
X_val,Y_val = f['val']['X'],np.copy(f['val']['Y'])

In [21]:
Y_train = dp.prepare_labels(Y_train)
Y_val = dp.prepare_labels(Y_val)

In [29]:
train_dataset = dp.HDF5Dataset(X_train,Y_train,train=True)
val_dataset = dp.HDF5Dataset(X_val,Y_val)

In [30]:
batch_size = 64

train_loader = DataLoader(train_dataset, num_workers=8, batch_size=batch_size, pin_memory=True,
                                                shuffle=True)
val_loader = DataLoader(val_dataset, num_workers=8, batch_size=batch_size, pin_memory=True,
                                                shuffle=True)

## Choosing the model

In [31]:
# Load pre-trained ResNet50 model
model = models.resnet50(weights='DEFAULT')
model_path = 'models/resnet50_retrained.pth'
loss_acc_path = 'models/resnet50_retrained.npz'
# Print the model architecture
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
model.fc = nn.Linear(in_features=2048, out_features=3)
model = model.to(device)

## Training

In [33]:
# Define the loss function
criterion = nn.CrossEntropyLoss().to(device)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.00005 ,weight_decay=1e-3)

#LR decay
decayRate = 0.96
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

In [34]:
n_total_steps = len(train_loader)
num_epochs = 20

max_val = 0.
PATH = os.path.join(dir,model_path)

train_loss,val_loss = np.zeros(num_epochs,dtype=np.float32),np.zeros(num_epochs,dtype=np.float32)
train_acc,val_acc = np.zeros(num_epochs,dtype=np.float32),np.zeros(num_epochs,dtype=np.float32)

for epoch in range(num_epochs):
    train_correct,train_samples = 0,0
    val_correct,val_samples = 0,0
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        # origin shape: [32, 3, 224, 224] = 32, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        train_loss[epoch] += loss.item()/len(train_loader)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        train_samples += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc[epoch] = 100.0 * train_correct / train_samples
    print('train loss %.4f - train acc. %.2f' %(train_loss[epoch],train_acc[epoch]))


    for val_images, val_labels in val_loader:
        val_images = val_images.to(device)
        val_labels = val_labels.type(torch.LongTensor)
        val_labels = val_labels.to(device)
        outputs = model(val_images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        val_samples += val_labels.size(0)
        val_correct += (predicted == val_labels).sum().item()

        val_loss[epoch] += criterion(outputs, val_labels).item()/len(val_loader)
    val_acc[epoch] = 100.0 * val_correct / val_samples
    if val_acc[epoch] > max_val:
        max_val = val_acc[epoch]
        torch.save(model.state_dict(), PATH)
    print('val loss %.4f - val acc. %.2f' %(val_loss[epoch],val_acc[epoch]))

    lr_scheduler.step()

print('Finished Training')

  0%|          | 0/312 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 312/312 [03:14<00:00,  1.60it/s]

train loss 0.7926 - train acc. 59.14


val loss 0.4704 - val acc. 82.75


100%|██████████| 312/312 [03:11<00:00,  1.63it/s]

train loss 0.6519 - train acc. 66.24


...

train loss 0.4791 - train acc. 74.51


val loss 0.2222 - val acc. 91.86


100%|██████████| 312/312 [03:11<00:00,  1.63it/s]

train loss 0.4793 - train acc. 74.32


val loss 0.2254 - val acc. 91.76
Finished Training


In [35]:
np.savez(os.path.join(dir,loss_acc_path),train_loss=train_loss,val_loss=val_loss,train_acc=train_acc,val_acc=val_acc)